# Import h36m data from 3d pose baseline

In [253]:
SUBJ_IDS = [1]

In [254]:
import data_utils

bpath = "/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m"
actions = ['Directions', 'Discussion', 'Eating', 'Greeting', 'Phoning', 'Photo', 'Posing', 'Purchases', 'Sitting', 'SittingDown', 'Smoking', 'Waiting', 'WalkDog', 'Walking', 'WalkTogether']
baseline_data = data_utils.load_data(bpath, SUBJ_IDS, actions)

Reading subject 1, action Directions
/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m/S1/MyPoses/3D_positions/Directions*.h5
/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m/S1/MyPoses/3D_positions/Directions 1.h5
/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m/S1/MyPoses/3D_positions/Directions.h5
Reading subject 1, action Discussion
/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m/S1/MyPoses/3D_positions/Discussion*.h5
/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m/S1/MyPoses/3D_positions/Discussion 1.h5
/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m/S1/MyPoses/3D_positions/Discussion.h5
Reading subject 1, action Eating
/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m/S1/MyPoses/3D_positions/Eating*.h5
/Users/Robert/Document

### Play with the data

In [255]:
print baseline_data.keys()
discussion1_key = next(k for k in baseline_data.keys() if k[1] == 'Discussion')
print (discussion1_key)

total = 0
for k in baseline_data.keys():
    total += len(baseline_data[k])
total

[(1, 'SittingDown', 'SittingDown.h5'), (1, 'Smoking', 'Smoking.h5'), (1, 'Directions', 'Directions.h5'), (1, 'WalkTogether', 'WalkTogether 1.h5'), (1, 'Eating', 'Eating.h5'), (1, 'Walking', 'Walking 1.h5'), (1, 'Greeting', 'Greeting.h5'), (1, 'Walking', 'Walking.h5'), (1, 'Purchases', 'Purchases.h5'), (1, 'Directions', 'Directions 1.h5'), (1, 'Purchases', 'Purchases 1.h5'), (1, 'Photo', 'Photo.h5'), (1, 'Sitting', 'Sitting 1.h5'), (1, 'WalkDog', 'WalkDog.h5'), (1, 'Phoning', 'Phoning.h5'), (1, 'Smoking', 'Smoking 1.h5'), (1, 'Photo', 'Photo 1.h5'), (1, 'Greeting', 'Greeting 1.h5'), (1, 'Discussion', 'Discussion 1.h5'), (1, 'Posing', 'Posing.h5'), (1, 'Waiting', 'Waiting 1.h5'), (1, 'Discussion', 'Discussion.h5'), (1, 'Eating', 'Eating 2.h5'), (1, 'Sitting', 'Sitting 2.h5'), (1, 'Waiting', 'Waiting.h5'), (1, 'WalkTogether', 'WalkTogether.h5'), (1, 'SittingDown', 'SittingDown 2.h5'), (1, 'Posing', 'Posing 1.h5'), (1, 'Phoning', 'Phoning 1.h5'), (1, 'WalkDog', 'WalkDog 1.h5')]
(1, 'Discus

62094

# Import h36m data from Matteo

In [256]:
import os
import json

HUMAN_ANNOTATION_DIR = '/Users/Robert/Documents/Caltech/CS81_Depth_Research/' \
                       'datasets/human36m_annotations'
HUMAN_ANNOTATION_FILE = 'human36m_train.json'

HUMAN_ANNOTATION_PATH = os.path.join(HUMAN_ANNOTATION_DIR, HUMAN_ANNOTATION_FILE)

with open(HUMAN_ANNOTATION_PATH) as f:
        _human_dataset = json.load(f)
        
_human_dataset['images'] = [d for d in _human_dataset['images'] if d['s_id'] in SUBJ_IDS]
_human_dataset['annotations'] = [d for d in _human_dataset['annotations'] if d['s_id'] in SUBJ_IDS]

### Play with the Data

In [257]:
print (_human_dataset['images'][0])
print (_human_dataset['annotations'][0])

# print (_human_dataset['actions'])

subject_ids = set()
for d in _human_dataset['images']:
    subject_ids.add(d['s_id'])

# print (subject_ids)

len(_human_dataset['images'])

{u'c_id': 55011271, u's_id': 1, u'frame': 0, u'height': 256, u'width': 256, u'video': u'Discussion 1.55011271.mp4', u'filename': u'human36m_train_0000790655.jpg', u'id': 790655}
{u'i_id': 790655, u's_id': 1, u'a_id': 0, u'kpts_2d': [125, 21, 125, 46, 144, 53, 107, 54, 160, 68, 95, 71, 148, 62, 104, 68, 145, 112, 110, 109, 142, 170, 116, 167, 148, 223, 116, 221], u'id': 790655, u'kpts_3d': [-212, -1245, 5677, -210, -1059, 5694, -69, -1006, 5704, -346, -996, 5676, 52, -857, 5502, -421, -830, 5464, -33, -865, 5265, -342, -816, 5226, -64, -563, 5752, -326, -579, 5708, -87, -121, 5734, -282, -148, 5799, -43, 283, 5935, -296, 268, 5980]}


12476

# Try and line up the data!
I'm going to try and line up Purchases first. If you notice, there are usually 3 action types in human dataset while the baseline dataset has 2. That's not a good sign they'll line up.

So upon further inspection, I figured it out. The baseline dataset is 5 times bigger than the human dataset. I believe it's because the human dataset skips every 20 frames of the video. Theoretically, this means the baseline dataset skips every 4 frames. I'll see if they line up now.

In [258]:
action = 'Discussion'

In [259]:
KEY_IND = 0

action_keys = [k for k in baseline_data.keys() if k[1] == action]
print (action_keys)
action_key = action_keys[KEY_IND]

baseline_data[action_key].shape

[(1, 'Discussion', 'Discussion 1.h5'), (1, 'Discussion', 'Discussion.h5')]


(3805, 96)

In [260]:
KEY_IND = 0

action_keys = [_act for _act in _human_dataset['actions'] if _act['name'] == action]
print (action_keys)
action_id = action_keys[KEY_IND]['id']

action_data = []
for d in _human_dataset['annotations']:
    if d['a_id'] == action_id:
        action_data.append(d['kpts_3d'])

len(action_data)

[{u'version': 1, u'id': 0, u'name': u'Discussion'}, {u'version': 2, u'id': 14, u'name': u'Discussion'}, {u'version': 0, u'id': 39, u'name': u'Discussion'}, {u'version': 3, u'id': 45, u'name': u'Discussion'}]


764

### After some ample visualization (which I cannot get to work here), I think the first picture lines up. Time to write a convertor function between things.

In [268]:
action = 'Discussion'
version = 0
i_human = 234
i_baseline = i_human * 5

# baseline
if version == 0:
    name = action + ".h5"
else:
    name = action + " " + str(version) + ".h5"
action_key = [k for k in baseline_data.keys() if k[2] == name][0]

print (baseline_data[action_key][i_baseline])


# human
action_key = [_act for _act in _human_dataset['actions'] if _act['name'] == action and _act['version'] == version][0]
action_id = action_key['id']

action_data_annotations = []
action_data_images = []
for d_ann, d_img in zip(_human_dataset['annotations'], _human_dataset['images']):
    if d_ann['a_id'] == action_id:
        action_data_annotations.append(d_ann)
        action_data_images.append(d_img)
# action_data_annotations = [d for d in _human_dataset['annotations'] if d['a_id'] == action_id]
# action_data_images = [d for d in _human_dataset['images'] if d['a_id'] == action_id]

action_data_annotations[i_human]['kpts_3d']

[ -466.10598755   133.38200378   911.40600586  -581.19702148    76.43810272
   945.85565186  -542.9644165    126.50431824   507.46395874  -538.84796143
   206.48236084    60.37329102  -454.88150024    72.18074036    22.87885094
  -420.07250977     5.9222765     27.68162346  -351.01477051   190.32600403
   876.95629883  -362.49658203   123.26842499   439.31842041  -515.26635742
   373.63430786    92.50153351  -421.02731323   253.01704407    37.15065002
  -373.60662842   195.58306885    28.34121513  -466.081604     133.39112854
   911.50256348  -496.5319519    254.2202301   1108.83691406  -537.91357422
   286.77539062  1360.46496582  -555.57354736   200.15466309  1443.28186035
  -584.22558594   261.84646606  1536.01098633  -537.91357422   286.77539062
  1360.46496582  -397.35290527   317.7467041   1314.69580078  -329.83068848
   187.89855957  1077.30200195  -312.58874512    75.65307617  1301.96484375
  -312.58874512    75.65307617  1301.96484375  -368.75357056    95.73480225
  1382.22802

[803,
 -1272,
 5045,
 779,
 -1078,
 5037,
 639,
 -1023,
 5038,
 917,
 -1031,
 5077,
 491,
 -890,
 5234,
 1049,
 -972,
 5315,
 457,
 -922,
 5482,
 1025,
 -978,
 5566,
 640,
 -583,
 5115,
 901,
 -618,
 5150,
 681,
 -168,
 5265,
 867,
 -187,
 5247,
 686,
 283,
 5306,
 867,
 265,
 5288]

### Looking into transforming the camera frame

In [271]:
import cameras

SUBJ_IDS = [1]
camera_frame = True
bpath = "/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m"
actions = ['Directions', 'Discussion', 'Eating', 'Greeting', 'Phoning', 'Photo', 'Posing', 'Purchases', 'Sitting', 'SittingDown', 'Smoking', 'Waiting', 'WalkDog', 'Walking', 'WalkTogether']
baseline_data = data_utils.load_data(bpath, SUBJ_IDS, actions)

if camera_frame:
    rcams = cameras.load_cameras("../data/h36m/cameras.h5", SUBJ_IDS)
    baseline_data = data_utils.transform_world_to_camera( baseline_data, rcams )
baseline_data

Reading subject 1, action Directions
/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m/S1/MyPoses/3D_positions/Directions*.h5
/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m/S1/MyPoses/3D_positions/Directions 1.h5
/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m/S1/MyPoses/3D_positions/Directions.h5
Reading subject 1, action Discussion
/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m/S1/MyPoses/3D_positions/Discussion*.h5
/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m/S1/MyPoses/3D_positions/Discussion 1.h5
/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m/S1/MyPoses/3D_positions/Discussion.h5
Reading subject 1, action Eating
/Users/Robert/Documents/Caltech/CS81_Depth_Research/models/3d-pose-baseline/data/h36m/S1/MyPoses/3D_positions/Eating*.h5
/Users/Robert/Document

{(1,
  'Directions',
  'Directions 1.54138969.h5'): array([[ -176.73076784,  -321.04861816,  5203.88206303, ...,
            12.12980163,  -175.03986102,  5510.04796057],
        [ -176.68641883,  -320.95474054,  5203.11211683, ...,
             8.40997288,  -177.15271427,  5511.04787716],
        [ -176.63382981,  -320.84309822,  5202.31378685, ...,
             5.64298043,  -178.60752146,  5511.08405639],
        ..., 
        [  -16.90275141,  -370.69514198,  5331.32994249, ...,
           206.5269549 ,  -196.00264729,  5594.56054215],
        [  -16.79412686,  -370.68475028,  5331.3652814 , ...,
           199.97929462,  -195.10661664,  5594.03793821],
        [  -16.73013124,  -370.73471292,  5331.45004222, ...,
           192.90153433,  -194.01031464,  5592.84375085]]),
 (1,
  'Directions',
  'Directions 1.55011271.h5'): array([[  225.83352776,  -503.33065834,  5567.39044354, ...,
          -105.80567312,  -301.75362652,  5557.06719967],
        [  226.35600907,  -503.50285025,  

### I figured out the camera frame makes the other set gross as well. Not as gross though. Perhaps I can't make my own rotation matrix? Explore this idea a bit

In [294]:
import cameras

cams = cameras.load_cameras("../data/h36m/cameras.h5", SUBJ_IDS)
ncams = 4

# Joints in H3.6M -- data has 32 joints, but only 17 that move; these are the indices.
H36M_NAMES = ['']*32
H36M_NAMES[0]  = 'Hip'
H36M_NAMES[1]  = 'RHip'
H36M_NAMES[2]  = 'RKnee'
H36M_NAMES[3]  = 'RFoot'
H36M_NAMES[6]  = 'LHip'
H36M_NAMES[7]  = 'LKnee'
H36M_NAMES[8]  = 'LFoot'
H36M_NAMES[12] = 'Spine'
H36M_NAMES[13] = 'Thorax'
H36M_NAMES[14] = 'Neck/Nose'
H36M_NAMES[15] = 'Head'
H36M_NAMES[17] = 'LShoulder'
H36M_NAMES[18] = 'LElbow'
H36M_NAMES[19] = 'LWrist'
H36M_NAMES[25] = 'RShoulder'
H36M_NAMES[26] = 'RElbow'
H36M_NAMES[27] = 'RWrist'

t3d_camera = {}
for t3dk in sorted( baseline_data.keys() ):

    subj, action, seqname = t3dk
    t3d_world = baseline_data[ t3dk ]

    for c in range( ncams ):
        R, T, f, c, k, p, name = cams[ (subj, c+1) ]
        print R
        camera_coord = cameras.world_to_camera_frame( np.reshape(t3d_world, [-1, 3]), R, T)
        camera_coord = np.reshape( camera_coord, [-1, len(H36M_NAMES)*3] )

        sname = seqname[:-3]+"."+name+".h5" # e.g.: Waiting 1.58860488.h5
        t3d_camera[ (subj, action, sname) ] = camera_coord



[[-0.91536173  0.40180837  0.02574754]
 [ 0.05154812  0.18037357 -0.98224649]
 [-0.39931903 -0.89778361 -0.18581953]]
[[ 0.92816834  0.37215384  0.00224838]
 [ 0.08166409 -0.1977723  -0.97684036]
 [-0.36309022  0.90685591 -0.21395759]]
[[-0.91415495 -0.40277802 -0.04572295]
 [-0.04562341  0.2143085  -0.97569994]
 [ 0.40278931 -0.88985489 -0.21428728]]
[[ 0.91415624 -0.40060706  0.06190599]
 [-0.05641001 -0.2769532  -0.95922617]
 [ 0.40141783  0.87339047 -0.27577674]]
[[-0.91536173  0.40180837  0.02574754]
 [ 0.05154812  0.18037357 -0.98224649]
 [-0.39931903 -0.89778361 -0.18581953]]
[[ 0.92816834  0.37215384  0.00224838]
 [ 0.08166409 -0.1977723  -0.97684036]
 [-0.36309022  0.90685591 -0.21395759]]
[[-0.91415495 -0.40277802 -0.04572295]
 [-0.04562341  0.2143085  -0.97569994]
 [ 0.40278931 -0.88985489 -0.21428728]]
[[ 0.91415624 -0.40060706  0.06190599]
 [-0.05641001 -0.2769532  -0.95922617]
 [ 0.40141783  0.87339047 -0.27577674]]
[[-0.91536173  0.40180837  0.02574754]
 [ 0.05154812  0.

KeyboardInterrupt: 